<a href="https://colab.research.google.com/github/j-buss/AutomateTheBoringStuffTutorial/blob/dev-chapter09/chapter11/imgurDownload.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Following the motivation of the "Automate the Boring Stuff" book by Al Sweigart. This is my own code to download images from an imgur gallery.

In [0]:
#! /usr/bin/python3

import requests
from bs4 import BeautifulSoup
import os
import logging

In [0]:
def returnSoup(pageURL, headers):
  req = requests.get(pageURL, headers)
  req.raise_for_status()
  soup = BeautifulSoup(req.text, "html.parser")
  return soup

In [0]:
imgurBaseURL = "https://imgur.com/search/score/all?q="
headers = requests.utils.default_headers()
headers.update({ 'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'})

In [0]:
gallerySoup = returnSoup(imgurBaseURL + "super mario brothers", headers)

In [0]:
def returnImgurGalleryLinks(soupObject):
    elems = soupObject.find_all("a",{"class":"image-list-link"})
    galleryLink = []
    logging.debug("Number of elements found: " + str(len(elems)))
   
    for i, link in enumerate(elems):
        tempLink = elems[i]['href']
        tempLink = 'https://imgur.com' + tempLink
        galleryLink.append(tempLink)
    
    return galleryLink

In [0]:
urlList = returnImgurGalleryLinks(gallerySoup)

In [19]:
urlList[0]

'https://imgur.com/gallery/VRg4v'

In [0]:
tempSoup = returnSoup(urlList[0], headers)

In [27]:
tempSoup

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd">

<html lang="en" xmlns="http://www.w3.org/1999/xhtml">
<head>
<link href="https://fonts.googleapis.com/css?family=Open+Sans:300,300i,400,400i,600,600i,700,700i,800,800i" rel="stylesheet"/>
<title>                        
        
    

    If Super Mario Brothers were made in 2013. - Album on Imgur</title>
<meta content="text/html;charset=utf-8" http-equiv="content-type"/>
<meta content="width=1138" name="viewport"/>
<meta content="follow, index" name="robots">
<meta content="images, photos, gif, gifs, memes, pictures, new pictures, reaction gifs, share photos, share images, latest images, funny, cute, visual storytelling, imgur" name="keywords"/>
<meta content="Post with 2148 votes and 58032 views. If Super Mario Brothers were made in 2013." name="description"/>
<meta content="Copyright 2019 Imgur, Inc." name="copyright"/>
<meta content="IE=Edge;" http-equiv="X-UA-Compatible"/

In [0]:
#elems = tempSoup.find_all("div",{"class":"post-image-container post-image-container--spacer"})
elems = tempSoup.find_all("script",{"type":"text/javascript"})
#script type="text/javascript"

In [31]:
len(elems)

19

In [0]:
for i in elems:
  if "mergeConfig" in i:
    print(i)

In [0]:
for u in urlList:
  tempSoup = returnSoup(u, headers)

In [0]:
def returnXkcdImageURL(soupObject):
  """For an Xkcd page; return the link to the image"""
  comicURL = "http:" + soupObject.find("div", {"id":"comic"}).find("img")['src']
  return comicURL

In [0]:
def downloadImage(imageURL, headers):
  """Download and image given the specific URL"""
  req = requests.get(imageURL, headers)
  req.raise_for_status()
  fileName = os.path.split(imageURL)[1]
  with open(fileName, 'wb') as f:
    for chunk in req.iter_content(100000):
      f.write(chunk)

In [0]:
def returnPrevURL(soupObject, baseURL):
  """Return the link to the previous Xkcd comic page"""
  previousLink = soupObject.find("div", {"id":"middleContainer"}).find("a",{"rel":"prev"})['href']
  previousLink = baseURL + previousLink
  return previousLink

In [0]:
def dailyXkcd(url, headers):
  """Perform the step to find the image url, download it, and return link to 
  previous page"""
  tempSoup = returnSoup(url, headers)
  try:
    imageURL = returnXkcdImageURL(tempSoup)
    downloadImage(imageURL, headers)
  except:
    print("Couldn't get Image from: " + url)
    pass
  prevURL = returnPrevURL(tempSoup, xkcdBaseURL)
  return prevURL

In [0]:
# Get Xkcd information for current webpage
prevURL = dailyXkcd(xkcdBaseURL,headers)
count = 0
while count < 50:
  prevURL = dailyXkcd(prevURL, headers)
  count += 1


Couldn't get Image from: https://xkcd.com/2198/
